# Set Up

In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from aws_sql_helper import *
import os
import json

In [2]:
from helper import *

In [3]:
# connect to the mongo client
client = connect_to_mongo()

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [4]:
# get the collection storing the problems
collection = get_collection(client = client,
    database_name = 'qa-repo',
    collection_name = 'qa')

# collection storing key : value
# key is 'problem_name'
# the value is the json containing all test cases

In [7]:
# get all problems' names from the collection
problems = set()
for problem in collection.find():
    problems.add(problem['problem_name'])

print(problems)

{'valid_palindrome_3', 'maximum_subarray', 'valid_palindrome', 'koko_eating_bananas', 'valid_palindrome_2', 'binary_search', 'contains_duplicate', 'longest_turbulent_subarray', 'two_sum', 'valid_parentheses'}


In [ ]:
aws_host, aws_port, aws_user, aws_password, aws_database = load_aws_connection_properties()
aws_connection, aws_cursor = connect_to_aws(aws_host, aws_port, aws_user, aws_password, aws_database)

### Set Difficulty - MUST DO
Caution, if multiple problems are being uploaded, this code will set ALL of them to the same difficulty set below.

In [ ]:
difficulty = 

# Upload Problem to Problem List and Test Cases

In [6]:
# Get the current directory where the script is located
current_directory = os.getcwd()

# Define the database directory (change this to your actual directory name)
database_directory = "test_cases_and_answers"

# Construct the full path to the database directory
database_path = os.path.join(current_directory, database_directory)

# Check if the directory exists
if os.path.exists(database_path) and os.path.isdir(database_path):

    # List all files in the directory
    files = os.listdir(database_path)
    
    # Print the list of files
    for file_name in files:

        file_path = os.path.join(database_path, file_name)
        print(file_name)

        problem_name = file_name.replace('.json', '')

        if os.path.isfile(file_path):

            if problem_name not in problems: # if the problem is NOT already in the database

                try:

                    with open(file_path, "r") as file:
                        
                        data = json.load(file)
                        data['problem_name'] = problem_name
                        inserted_object = collection.insert_one(data)
                        print(f"Uploaded file: {file_path}, Object ID: {inserted_object.inserted_id}")

                    add_entry_to_table(aws_connection, aws_cursor, problem_name, difficulty)

                except Exception as e:
                    print(f"Error uploading file {file_path}: {str(e)}")

else:
    print(f"Database directory '{database_path}' does not exist.")

valid_parentheses.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/valid_parentheses.json, Object ID: 65473db109e320bc340ea7aa
maximum_subarray.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/maximum_subarray.json, Object ID: 65473db109e320bc340ea7ab
two_sum.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/two_sum.json, Object ID: 65473db209e320bc340ea7ac
binary_search.json
valid_palindrome_3.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/valid_palindrome_3.json, Object ID: 65473db209e320bc340ea7ad
valid_palindrome_2.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/valid_palindrome_2.json, Object ID: 65473db209e320bc340ea7ae
longest_turbulent_s

# Upload Problem Markdown

In [9]:
def read_markdown_files_and_upload(folder_path, collection_object):

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.md'):
            with open(os.path.join(folder_path, file_name), 'r') as file:
                markdown_content = file.read()

            # Create the JSON object
            json_object = {
                "problem_name": os.path.splitext(file_name)[0],
                "markdown": markdown_content
            }

            # Check if the problem already exists in the collection
            if collection_object.find_one({"problem_name": json_object["problem_name"]}):
                print(f"Skipped {file_name} as the problem already exists in the collection.")
                continue

            # Upload the object to the MongoDB collection
            try:
                collection_object.insert_one(json_object)
                print(f"Uploaded {file_name} to the collection.")
            except Exception as e:
                print(f"Error uploading {file_name}: {e}")

In [10]:
# Get the current directory where the script is located
current_directory = os.getcwd()

# Define the database directory (change this to your actual directory name)
markdown_directory = "questions"

# Construct the full path to the database directory
markdown_path = os.path.join(current_directory, markdown_directory)

In [11]:
# get the collection storing the problems
md_collection = get_collection(client = client,
    database_name = 'qa-repo',
    collection_name = 'markdown_repo')

In [13]:
# Example usage
collection_name = 'markdown_repo'  # Replace with your collection name

read_markdown_files_and_upload(markdown_path, md_collection)

Skipped valid_parentheses.md as the problem already exists in the collection.
Skipped longest_turbulent_subarray.md as the problem already exists in the collection.
Skipped two_sum.md as the problem already exists in the collection.
Skipped koko_eating_bananas.md as the problem already exists in the collection.
Skipped valid_palindrome.md as the problem already exists in the collection.
Skipped contains_duplicate.md as the problem already exists in the collection.
Skipped maximum_subarray.md as the problem already exists in the collection.
Skipped binary_search.md as the problem already exists in the collection.
Skipped valid_palindrome_3.md as the problem already exists in the collection.
Skipped valid_palindrome_2.md as the problem already exists in the collection.


In [14]:
# print out the mardown of problem "binary_search"
print(md_collection.find_one({"problem_name": "binary_search"})['markdown'])

# Binary Search

Given an array of integers nums which is sorted in ascending order, and an integer target, write a function to search target in nums. If target exists, then return its index. Otherwise, return -1.

You must write an algorithm with O(log n) runtime complexity.

## Example 1:

    Input: nums = [-1,0,3,5,9,12], target = 9
    Output: 4
    Explanation: 9 exists in nums and its index is 4

## Example 2:

    Input: nums = [-1,0,3,5,9,12], target = 2
    Output: -1
    Explanation: 2 does not exist in nums so return -1

## Constraints:

    1 <= nums.length <= 104
    -104 < nums[i], target < 104
    All the integers in nums are unique.
    nums is sorted in ascending order.
